# scikit naive bayes

15 June 2017. 양철웅



`scikit-learn` 라이브러리를 이용하여 텍스트 파일에 대한 카테고리 분석을 진행해본다.
Naive Bayes 분석을 이용하는데 왜냐하면 (1) 텍스트 분석에서 Naive Bayes는 웬만한 Deep Learning보다 더 효율적이며, (2) 또한 그로 인해 baseline 알고리즘으로써의 역할을 하기 때문이다. Naive Bayes분석보다 못 한 알고리즘은 버려야한다.

이 문서는 http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html 를 나름대로 정리한 것이다.

## 준비사항

### 라이브러리 설치
우선 `scikit-learn` 라이브러리를 설치한다. 필요에 따라서 `libblas-dev liblapack-dev gfortran`를 먼저 설치해야 할 수 있다.
```
pip install -U scikit-learn[alldeps]
```

### 데이터셋
20,000 뉴스그룹 문서로 이루어진 데이터셋인 "Twenty Newsgroups"을 사용한다. 20개의 서로 다른 뉴스그룹에서 발췌한 문서이다. 이 데이터셋은 텍스트 분류 및 클러스터링등의 기계학습 실험에서 많이 사용되었다. 해당 데이터셋은 API를 이용하여 자동으로 다운받을 수 있다. http://qwone.com/~jason/20Newsgroups/ 에서 수동으로 다운받을 수도 있다.

## 코드
### 데이터셋 읽기

In [38]:
from sklearn.datasets import fetch_20newsgroups
# 우선 테스트로 4개의 뉴스그룹만 사용
# categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']
categories = None # all group

# 해당 카테고리에 매칭되는 파일을 읽어온다. 읽어올 때 shuffle을 수행한다.
twenty_train = fetch_20newsgroups(subset='train',
                categories=categories, shuffle=True, random_state=42)

print "dataset length=%d" % len(twenty_train.data)
print "target_names=%s" % twenty_train.target_names
#first four lines of the first record
print "\n".join(twenty_train.data[0].split("\n")[:3])
#target (==y). 해당 레코드가 속한 카테고리 id이다. 이를 이용하여 supervised learning을 진행한다.
print twenty_train.target_names[twenty_train.target[0]]
print twenty_train.target[0:10]


dataset length=11314
target_names=['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
rec.autos
[ 7  4  4  1 14 16 13  3  2  4]


### 텍스트 전처리 
BoW (Bag of words)를 만든다. 방법은 다음과 같다.
- 각 단어로 dictionary를 만든다.
- 각 문서 `#i`에 대하여 각 단어 `w`의 빈도수를 세어 그 수를 `X[i,j]`에 저장한다. `j`는 `w`의 dictionary index이다.

`n_feature`는 모든 단어의 수이며 100K을 넘게 된다. 문서의 수가 10K를 넘는 경우 `X`의 저장을 위해서는 1G \* 4 bytes (float32) = __4GB__ 메모리를 요구한다. 

`X`를 sparse 자료구조를 이용하면 메모리 소모양이 감소한다. 따라서 `scipy.sparse` 행렬을 사용한다.

영문 stopword (a,the,..)등의 제거, 토큰화, n-gram기능등을 위해 `scikit-learn`의 `CountVectorizer`를 이용한다.


In [39]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words=None) # None, 'english', or custom list
# Learn the vocabulary dictionary and return term-document matrix
X_train_counts = count_vect.fit_transform(twenty_train.data)

print X_train_counts.shape  # 문서수 x 총단어수
print count_vect.vocabulary_.get(u'algorithm') # 'algorithm'의 출현횟수

# occurrence => term frequency
# 또한 많이 나오는 term에 대해서는 weight를 줄인다: TF-IDF: Term Frequency x Inverse Docuement Frequency
# idf 안쓰고 싶으면 TfidfTransformer(use_idf=False)를 쓴다.
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape
print len(twenty_train.target)

(11314, 130107)
27366
(11314, 130107)
11314


### 학습
multinomial (멀티 카테고리) naive bayes를 이용하여 학습해보자

In [40]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target) # classifier

# test
def test1(clf):
    foo = ['There is no god in the world', 'OpenGL runs on GPU', 'aspirin is a cheap pill']
    X_foo_counts = count_vect.transform(foo)
    X_foo_tfidf = tfidf_transformer.transform(X_foo_counts)
    predicted = clf.predict(X_foo_tfidf)
    for doc, category in zip(foo, predicted):
        print '%r => %s' % (doc, twenty_train.target_names[category])
# test1(clf)

# pipelining vectorizer => transformer => classifier
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB())])
_ = text_clf.fit(twenty_train.data, twenty_train.target)

### 테스트

In [41]:
import numpy as np
# training set이 아닌 별도의 test set을 가지고 검증한다.
twenty_test = fetch_20newsgroups(subset='test',
                categories = categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
print "naive bayes accuracy = %f" % np.mean(predicted == twenty_test.target)


naive bayes accuracy = 0.773898


### 통계
- precision = True Positive / (True Positive + False Positive). 검색된 문서들 중 관련있는 문서의 비율
- recall = True Positive / (True Positive + False Negative). 관련있는 문서들 중 검색된 문서의 비율
- f1-score = 2 * precision * recall / (precision + recall). precision과 recall의 조화평균
- support = The number of occurrences of each label in y_true
- confusion matrix - 실제 클래스(row)와 예측클래스(column)간의 관계이다. 아래를 보면 기사들이 꽤 sci.crypt, soc.religion.christian으로 오분류되었다

In [45]:
from sklearn import metrics
print metrics.classification_report(twenty_test.target, predicted,
                                    target_names=twenty_test.target_names)
print metrics.confusion_matrix(twenty_test.target, predicted)

                          precision    recall  f1-score   support

             alt.atheism       0.80      0.52      0.63       319
           comp.graphics       0.81      0.65      0.72       389
 comp.os.ms-windows.misc       0.82      0.65      0.73       394
comp.sys.ibm.pc.hardware       0.67      0.78      0.72       392
   comp.sys.mac.hardware       0.86      0.77      0.81       385
          comp.windows.x       0.89      0.75      0.82       395
            misc.forsale       0.93      0.69      0.80       390
               rec.autos       0.85      0.92      0.88       396
         rec.motorcycles       0.94      0.93      0.93       398
      rec.sport.baseball       0.92      0.90      0.91       397
        rec.sport.hockey       0.89      0.97      0.93       399
               sci.crypt       0.59      0.97      0.74       396
         sci.electronics       0.84      0.60      0.70       393
                 sci.med       0.92      0.74      0.82       396
         

### 참고: SVM(서포트 벡터 머신) classifier와의 비교


In [48]:
from sklearn.linear_model import SGDClassifier
# SGDClassifier: Stochastic Gradient Descent Classifier
# This implementation works with data represented as dense or sparse arrays of 
# floating point values for the features. The model it fits can be controlled 
# with the loss parameter; by default, it fits a linear support vector machine (SVM).
svm_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                    ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                          alpha=1e-3, n_iter=5,
                                          random_state=42))])
_ = svm_clf.fit(twenty_train.data, twenty_train.target)
predicted_svm = svm_clf.predict(docs_test)
print "SVM accuracy = %f" % np.mean(predicted_svm == twenty_test.target)

print metrics.classification_report(twenty_test.target, predicted_svm,
                                    target_names=twenty_test.target_names)

SVM accuracy = 0.823818
                          precision    recall  f1-score   support

             alt.atheism       0.73      0.72      0.72       319
           comp.graphics       0.80      0.70      0.74       389
 comp.os.ms-windows.misc       0.73      0.76      0.75       394
comp.sys.ibm.pc.hardware       0.71      0.70      0.70       392
   comp.sys.mac.hardware       0.83      0.81      0.82       385
          comp.windows.x       0.83      0.77      0.80       395
            misc.forsale       0.84      0.90      0.87       390
               rec.autos       0.92      0.89      0.91       396
         rec.motorcycles       0.92      0.96      0.94       398
      rec.sport.baseball       0.89      0.90      0.89       397
        rec.sport.hockey       0.88      0.99      0.93       399
               sci.crypt       0.83      0.96      0.89       396
         sci.electronics       0.83      0.60      0.70       393
                 sci.med       0.87      0.86      

### 참고2: 그리드 검색을 이용한 하이퍼파라메터 튜닝
학습시에 학습 파라메터(hyperparameter)를 여러가지로 변경하면서 최적값을 찾아야 하는데, sci-kit에서는 그를 위하여 그리드 검색을 제공한다. 하이퍼파라메터별로 값들을 지정하면 그 조합대로 테스트를 해 보며 최적값을 찾아준다.
여기서는 SVM classifier를 이용할 경우, ngram값 (1-gram, 2-gram), idf유무, alpha값의 조합을 테스트하는 예를 들어본다. 모두 2\*2\*2 = 8개의 조합을 테스트하게된다.

In [53]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1,1), (1,2)],
              'tfidf__use_idf': (True, False),
              'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(svm_clf, parameters, n_jobs=-1) # n_jobs is the number of CPU cores
_ = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])

print "\"God is love\" is group %s" % twenty_train.target_names[gs_clf.predict(['God is love'])[0]]
# object's `best_core_` and `best_params_` attributes store the best mean score 
# and the parameter settings
print gs_clf.best_score_
for param_name in sorted(parameters.keys()):
    print "%s: %r" % (param_name, gs_clf.best_params_[param_name])

"God is love" is group soc.religion.christian
0.6025
clf__alpha: 0.001
tfidf__use_idf: True
vect__ngram_range: (1, 1)
